In [1]:
import numpy as np
import os
#import struct
#import itertools
import matplotlib.pyplot as plt

import jax, optax
from jax import random, numpy as jnp
from flax import linen as nn
from flax.training import train_state
from flax.core.frozen_dict import freeze, unfreeze

!git clone https://github.com/htymjun/da-sr
os.chdir('/content/da-sr')

from read import fw_to_np

Cloning into 'da-sr'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 19 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (19/19), 9.97 MiB | 10.69 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [2]:
u, v, w, p, t, icount, x, y, z = fw_to_np('data')

In [3]:
# check size
print(np.shape(u))
print(np.shape(v))
print(np.shape(w))
print(np.shape(p))

print(np.shape(x))
print(np.shape(y))
print(np.shape(z))

(201, 100, 20)
(200, 101, 20)
(200, 100, 21)
(200, 100, 20)
(201,)
(101,)
(21,)


In [4]:
# interpolate
uc = 0.5*(u[:-1,:,:]+u[1:,:,:])
vc = 0.5*(v[:,:-1,:]+v[:,1:,:])
wc = 0.5*(w[:,:,:-1]+w[:,:,1:])
xc = 0.5*(x[:-1]+x[1:])
yc = 0.5*(y[:-1]+y[1:])
zc = 0.5*(z[:-1]+z[1:])

In [ ]:


fig, ax = plt.subplots()
tcf = ax.contourf(xc, yc, p)
fig.colorbar(tcf)

In [ ]:
filter0 = np.random.standard_normal((5,5,5))
filter1 = np.random.standard_normal((5,5,5))

filter_array = np.zeros([5, 5, 5, 2])
filter_array[:, :, :, 0] = filter0
filter_array[:, :, :, 1] = filter1

In [ ]:
class FixedConvFilterModel(nn.Module):
    @nn.compact
    def __call__(self, x, apply_pooling=False):
        Nx = 10
        Ny = 10
        Nz = 10
        x = x.reshape([-1, Nx, Ny, Nz, 1])
        x = nn.Conv(features=2, kernel_size=(5, 5, 5), use_bias=False)(x)
        x = jnp.abs(x)
        x = nn.relu(x-0.2)
        if apply_pooling:
            x = nn.max_pool(x, window_shape=(2, 2, 2), strides=(2, 2, 2))
        return x

In [ ]:
key, key1 = random.split(random.PRNGKey(0))
variables = FixedConvFilterModel().init(key1, images[0:1])

jax.tree_util.tree_map(lambda x: x.shape, variables['params'])

In [ ]:
params = unfreeze(variables['params'])
params['Conv_0']['kernel'] = jnp.asarray(filter_array)
new_params = freeze(params)

state = train_state.TrainState.create(
    apply_fn=FixedConvFilterModel().apply,
    params=new_params,
    tx=optax.adam(learning_rate=0.001))

In [ ]:
conv_output = jax.device_get(
    state.apply_fn({'params': state.params}, images[:]))
filter_vals = jax.device_get(state.params['Conv_0']['kernel'])